# Lab 17: Foundry Local - ISS Daily Reports Chatbot

## The Scenario

You're an astronaut aboard the International Space Station. You want to query historical ISS mission reports, but there's a problem: communication with Earth has a 2-second delay, bandwidth is limited, and you'd rather not send every question through NASA's ground control.

**Solution**: Run an LLM directly on your laptop using **Foundry Local**. No internet needed for inference. Your questions stay private. Responses are instant.

This lab demonstrates how to build a fully local AI assistant that fetches real NASA ISS Daily Summary Reports.

| Concept | Description |
|---------|-------------|
| Foundry Local | Run LLMs directly on your device via ONNX Runtime |
| Function Calling | AI calls Python functions to fetch real data |
| NASA Data | Real ISS reports from March 2013 to July 2024 |

## Step 1: Install Dependencies

In [23]:
import platform, os, subprocess, shutil
from IPython.display import display, HTML

def show_status(title: str, message: str, status: str = "info"):
    colors = {
        "success": ("#4CAF50", "#81C784"),
        "error": ("#F44336", "#E57373"),
        "warning": ("#FF9800", "#FFB74D"),
        "info": ("#2196F3", "#64B5F6"),
    }
    border, text = colors.get(status, colors["info"])
    display(HTML(f"""
    <div style="border-left: 4px solid {border}; padding: 8px 12px; margin: 6px 0;">
        <span style="color: {text}; font-weight: 600;">{title}:</span>
        <span style="color: #B0B0B0;"> {message}</span>
    </div>"""))

# Check OS
current_os = platform.system()
os_display = {"Darwin": "macOS", "Windows": "Windows"}.get(current_os, current_os)
show_status("OS", os_display, "info")

if current_os not in ["Darwin", "Windows"]:
    show_status("Error", "Foundry Local requires macOS or Windows", "error")
    raise SystemError(f"Unsupported: {current_os}")

# Check devcontainer
if any([os.environ.get("REMOTE_CONTAINERS"), os.environ.get("CODESPACES"), 
        os.path.exists("/.dockerenv"), os.environ.get("DEVCONTAINER")]):
    show_status("Error", "Please run on host system, not in container", "error")
    raise SystemError("Container detected")

# Check Foundry Local
if not shutil.which("foundry"):
    cmd = "brew tap microsoft/foundry-local && brew install foundry-local" if current_os == "Darwin" else "winget install Microsoft.FoundryLocal"
    show_status("Install Foundry Local", f"<code>{cmd}</code>", "warning")
    raise SystemError("Foundry Local not found")

version = subprocess.run(["foundry", "--version"], capture_output=True, text=True).stdout.strip()
show_status("Foundry Local", version or "installed", "success")

In [24]:
# Install required packages
!pip install openai foundry-local-sdk -q

## Step 2: Initialize Foundry Local

First run downloads the model (~2GB). Recommended: `phi-4-mini` for best function calling.

In [25]:
import openai
from foundry_local import FoundryLocalManager
from display_helpers import show_welcome, show_model_loading, show_model_ready

show_welcome()

MODEL_ALIAS = "phi-4-mini"
show_model_loading(MODEL_ALIAS)

manager = FoundryLocalManager(MODEL_ALIAS)
model_info = manager.get_model_info(MODEL_ALIAS)
MODEL_ID = model_info.id

show_model_ready(MODEL_ID, manager.endpoint)


# ISS Daily Reports Chatbot

**Foundry Local** powered assistant - runs entirely locally using ONNX Runtime.


Loading model: **phi-4-mini** (first run may take a few minutes to download)

Model ready: `Phi-4-mini-instruct-generic-gpu:5` at `http://127.0.0.1:51137/v1`

## Step 3: Create OpenAI Client

Foundry Local provides an OpenAI-compatible endpoint - standard SDK just works.

In [26]:
client = openai.OpenAI(
    base_url=manager.endpoint,
    api_key=manager.api_key
)
print(f"Client ready: {manager.endpoint}")

Client ready: http://127.0.0.1:51137/v1


## Step 4: Load Tools

The AI can call `get_report_by_date(date)` to fetch reports from NASA's website.

In [27]:
from iss_helpers import FOUNDRY_LOCAL_TOOLS, execute_function, parse_foundry_local_response

for tool in FOUNDRY_LOCAL_TOOLS:
    print(f"Tool: {tool['name']} - {tool['description']}")

Tool: get_report_by_date - Fetch the real ISS Daily Summary Report from NASA for a specific date. Data available from March 2013 to July 29, 2024. Use YYYY-MM-DD format.


## Step 5: Test a Question

In [28]:
from display_helpers import (
    show_user_message, show_assistant_message, show_function_call, 
    show_function_result_preview, show_no_function_call
)

def chat_with_iss_assistant(user_message: str):
    """Send a message to the ISS assistant and get a response."""
    show_user_message(user_message)
    
    system_prompt = """You are an ISS Daily Reports assistant. You MUST use function calling to get data.

AVAILABLE FUNCTION:
- get_report_by_date: Fetch ISS report for a date (March 2013 - July 2024)

CRITICAL: When the user asks about any date, respond with ONLY:
functools[{"name": "get_report_by_date", "arguments": {"date": "YYYY-MM-DD"}}]

DO NOT write code. DO NOT explain. Just output the functools line."""
    
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_message}
    ]
    
    response = client.chat.completions.create(
        model=MODEL_ID,
        messages=messages,
        tools=FOUNDRY_LOCAL_TOOLS,
        temperature=0.0,
        max_tokens=256
    )
    
    content = response.choices[0].message.content or ""
    function_calls = parse_foundry_local_response(content)
    
    if not function_calls:
        show_no_function_call()
        show_assistant_message(content)
        return content
    
    # Execute function calls
    all_results = []
    for func_call in function_calls:
        name = func_call.get("name", "")
        args = func_call.get("arguments", {})
        
        if name:
            show_function_call(name, args)
            result = execute_function(name, args)
            all_results.append({"function": name, "result": result})
            show_function_result_preview(result)
    
    # Generate summary from results (truncate to fit context)
    if all_results:
        # Limit result size to avoid context overflow
        results_text = all_results[0]["result"][:2000]
        
        final_response = client.chat.completions.create(
            model=MODEL_ID,
            messages=[
                {"role": "system", "content": "Summarize this ISS report concisely."},
                {"role": "user", "content": results_text}
            ],
            temperature=0.3,
            max_tokens=512
        )
        
        final_content = final_response.choices[0].message.content or ""
        show_assistant_message(final_content)
        return final_content
    
    return content

# Test with a question
chat_with_iss_assistant("What happened on the ISS on July 18, 2024?")

'The ISS Daily Summary Report for July 18, 2024, highlights various experiments and maintenance activities. Key experiments include the Electro-static Levitation Furnace (ELF), involving sample handling and stowing for return. The crew also engaged in Ham radio contact and installed Nano Particle Haloing Suspension hardware. Medical experiments included ultrasound scans, OCT, and blood pressure measurements for the Thigh Cuff investigation. Educational efforts featured a video on DNA structure and function. Systems activities included ongoing food inventory audits and recharging of Power Tool Battery packs. All experiments and tasks were successfully completed. [Source: NASA Blog]'

## Step 6: Local Models vs Foundation Models

Local models like `phi-4-mini` may not match cloud-based foundation models (GPT-5, Claude, etc.), but they are **more than sufficient** for Q&A and summarization tasks.

| Use Case | Local Model Performance |
|----------|------------------------|
| Q&A over data | Excellent |
| Summarization | Very Good |
| Function calling | Good |

**Key advantages**: Privacy, low latency, no API costs, works offline.

Let's demonstrate summarization by fetching all ISS reports from a week and generating a summary.

In [ ]:
from datetime import datetime, timedelta
from iss_helpers import get_report_by_date
from display_helpers import show_user_message, show_assistant_message

def generate_weekly_summary(year: int, month: int, day: int):
    """Fetch all reports for a week and generate a summary using the local model."""
    start_date = datetime(year, month, day)
    end_date = start_date + timedelta(days=6)
    week_str = f"week of {start_date.strftime('%B %d, %Y')}"
    
    show_user_message(f"Generate a summary of ISS activities for {week_str}")
    print(f"Fetching reports from {start_date.date()} to {end_date.date()}...")
    
    # Fetch weekday reports
    all_reports = []
    current_date = start_date
    while current_date <= end_date:
        if current_date.weekday() < 5:  # Skip weekends
            date_str = current_date.strftime("%Y-%m-%d")
            result = get_report_by_date(date_str)
            if "success" in result and '"success": true' in result.lower():
                all_reports.append({"date": date_str, "content": result})
                print(f"  {date_str}: OK")
        current_date += timedelta(days=1)
    
    print(f"\nFetched {len(all_reports)} reports. Generating summary...")
    
    if not all_reports:
        print("No reports found.")
        return
    
    # Combine reports
    combined_text = "\n\n".join([f"**{r['date']}**:\n{r['content'][:1500]}" for r in all_reports])
    
    response = client.chat.completions.create(
        model=MODEL_ID,
        messages=[
            {"role": "system", "content": "Summarize ISS mission reports concisely."},
            {"role": "user", "content": f"Summarize these ISS reports from {week_str}:\n\n{combined_text}"}
        ],
        temperature=0.1,
        max_tokens=512
    )
    
    summary = response.choices[0].message.content
    show_assistant_message(summary)
    return summary

# Generate summary for week of July 20, 2024
summary = generate_weekly_summary(2024, 7, 20)

Fetching reports from 2023-07-01 to 2023-07-31...
  2023-07-10: OK
  2023-07-11: OK
  2023-07-12: OK
  2023-07-13: OK
  2023-07-14: OK
  2023-07-18: OK
  2023-07-19: OK
  2023-07-20: OK
  2023-07-21: OK
  2023-07-24: OK
  2023-07-25: OK
  2023-07-26: OK
  2023-07-27: OK
  2023-07-28: OK
  2023-07-31: OK

Fetched 15 reports. Generating summary...


KeyboardInterrupt: 

## Summary

Local models like `phi-4-mini` are highly capable for Q&A and summarization, with key advantages: **privacy**, **low latency**, **no API costs**, and **offline availability**.

| Component | Description |
|-----------|-------------|
| Foundry Local | Runs LLMs locally via ONNX Runtime |
| NASA Data | Real ISS reports (March 2013 - July 2024) |
| Function Calling | AI decides when to fetch data |